# OpenAI APIs - Vision

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/vision).
This tutorial covers the vision APIs for vision language models.

SGLang supports various vision language models such as Llama 3.2, LLaVA-OneVision, Qwen2.5-VL, Gemma3 and [more](../supported_models/multimodal_language_models.md).

As an alternative to the OpenAI API, you can also use the [SGLang offline engine](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py).

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

vision_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 00:32:24.692000 2997830 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 00:32:24.692000 2997830 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 00:32:26] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=31675, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7835956249999999, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_s

[2025-08-17 00:32:27] Ignore import error when loading sglang.srt.multimodal.processors.glm4v: No module named 'transformers.models.glm4v_moe'


You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
[2025-08-17 00:32:27] You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


[2025-08-17 00:32:28] Using default HuggingFace chat template with detected content format: openai


...

Capturing batches (bs=1 avail_mem=23.08 GB): 100%|██████████| 3/3 [00:00<00:00,  3.82it/s]
[2025-08-17 00:32:45] Capture cuda graph end. Time elapsed: 1.29 s. mem usage=0.13 GB. avail mem=23.05 GB.


[2025-08-17 00:32:46] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=128000, available_gpu_mem=23.05 GB


[2025-08-17 00:32:46] INFO:     Started server process [2997830]
[2025-08-17 00:32:46] INFO:     Waiting for application startup.
[2025-08-17 00:32:46] INFO:     Application startup complete.
[2025-08-17 00:32:46] INFO:     Uvicorn running on http://127.0.0.1:31675 (Press CTRL+C to quit)


[2025-08-17 00:32:47] INFO:     127.0.0.1:38418 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-17 00:32:47] INFO:     127.0.0.1:38432 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 00:32:47] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:32:48] INFO:     127.0.0.1:38434 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 00:32:48] The server is fired up and ready to roll!


## Using cURL

Once the server is up, you can send test requests using curl or requests.

In [2]:
import subprocess

curl_command = f"""
curl -s http://localhost:{port}/v1/chat/completions \\
  -H "Content-Type: application/json" \\
  -d '{{
    "model": "Qwen/Qwen2.5-VL-7B-Instruct",
    "messages": [
      {{
        "role": "user",
        "content": [
          {{
            "type": "text",
            "text": "What’s in this image?"
          }},
          {{
            "type": "image_url",
            "image_url": {{
              "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
            }}
          }}
        ]
      }}
    ],
    "max_tokens": 300
  }}'
"""

response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)


response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)

[2025-08-17 00:32:53] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:32:54] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, cuda graph: True, gen throughput (token/s): 5.02, #queue-req: 0, 


[2025-08-17 00:32:54] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, cuda graph: True, gen throughput (token/s): 112.34, #queue-req: 0, 
[2025-08-17 00:32:54] INFO:     127.0.0.1:38436 - "POST /v1/chat/completions HTTP/1.1" 200 OK


[2025-08-17 00:32:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 306, token usage: 0.01, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:32:55] Decode batch. #running-req: 1, #token: 345, token usage: 0.02, cuda graph: True, gen throughput (token/s): 44.37, #queue-req: 0, 


[2025-08-17 00:32:56] INFO:     127.0.0.1:59326 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "Qwen/Qwen2.5-VL-7B-Instruct",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    "max_tokens": 300,
}

response = requests.post(url, json=data)
print_highlight(response.text)

[2025-08-17 00:32:56] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 306, token usage: 0.01, #running-req: 0, #queue-req: 0, 
[2025-08-17 00:32:56] Decode batch. #running-req: 1, #token: 315, token usage: 0.02, cuda graph: True, gen throughput (token/s): 47.97, #queue-req: 0, 


[2025-08-17 00:32:57] Decode batch. #running-req: 1, #token: 355, token usage: 0.02, cuda graph: True, gen throughput (token/s): 78.88, #queue-req: 0, 


[2025-08-17 00:32:57] Decode batch. #running-req: 1, #token: 395, token usage: 0.02, cuda graph: True, gen throughput (token/s): 81.31, #queue-req: 0, 
[2025-08-17 00:32:57] INFO:     127.0.0.1:59332 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-7B-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)

[2025-08-17 00:32:57] Prefill batch. #new-seq: 1, #new-token: 292, #cached-token: 15, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:32:58] Decode batch. #running-req: 1, #token: 343, token usage: 0.02, cuda graph: True, gen throughput (token/s): 42.08, #queue-req: 0, 


[2025-08-17 00:32:58] Decode batch. #running-req: 1, #token: 383, token usage: 0.02, cuda graph: True, gen throughput (token/s): 79.47, #queue-req: 0, 
[2025-08-17 00:32:59] INFO:     127.0.0.1:59334 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Multiple-Image Inputs

The server also supports multiple images and interleaved text and images if the model supports it.

In [5]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-7B-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ],
    temperature=0,
)

print_highlight(response.choices[0].message.content)

[2025-08-17 00:32:59] Prefill batch. #new-seq: 1, #new-token: 2532, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 00:33:00] Decode batch. #running-req: 1, #token: 2574, token usage: 0.13, cuda graph: True, gen throughput (token/s): 30.14, #queue-req: 0, 
[2025-08-17 00:33:00] INFO:     127.0.0.1:59346 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(vision_process)